In [1]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt

# Power Method and Lanczos Algorithm

## Power Method

This is an algorithm fo find the dominant eigenvalue (largest in magnitude) and its corresponding eigenvector of a square matrix.

To find the eigenvector corresponding to the dominant eigenvalue for matrix M:
1) Randomly choose an initial guess for the eigenvector $\mathbf{v}_0$   
2) Compute $\mathbf{v}_0=M\mathbf{v}_0$   
3) Normalize $\mathbf{v}_0$   
If the result converges, break out. If not, continue from step 2.

In [2]:
def power_method(M , max_iter = 1e6):
    
    n = M.shape[1]
    x = np.random.random(n)
    
    # Go until the gap between two iterations is less than a tolerance
    tol = 1e-10
    gap = 1
    
    cur_iter = 0
    
    while gap>tol:
        
        if cur_iter > max_iter:
           break
        
        old_x = x
        x = np.dot(M,x)
        x = x / LA.norm(x)
        gap = abs(LA.norm(x - old_x))
        cur_iter +=1
    
    print("The eigenvector of the largest eigenvalue is : ", x)
    return x

In [3]:
M = np.array([ [1,0,0], [0,1,0], [0,0,1]])
x = power_method(M)

The eigenvector of the largest eigenvalue is :  [0.43672143 0.68942623 0.5778978 ]


In [4]:
M = np.array([ [5,-3,-3], [0,-4,-3], [0,6,5]])
x = power_method(M)

The eigenvector of the largest eigenvalue is :  [-1.00000000e+00 -2.46161097e-11  4.92322191e-11]


In [5]:
# Get the largest eigenvalue by
np.dot(M,x)/x

array([5.        , 1.99999996, 1.99999998])

Apparently the largest one is the largest eigenvalue.

In [6]:
# Check the result by this function
LA.eig(M)

(array([ 5., -1.,  2.]),
 array([[ 1.        ,  0.        , -0.40824829],
        [ 0.        , -0.70710678,  0.40824829],
        [ 0.        ,  0.70710678, -0.81649658]]))

## Lanczos Algorithm (Simplified)

This algorithm will find the eigenvectors in order of the size of the absolute value of the eigenvalue.

To find the eigenvector corresponding to the $k$ th largest eigenvalue:
1) Start with a randomly chosen vector $\mathbf{x}_0$       
2) Replace $\mathbf{x}_0$ with its projection of previously found eigenvectors

$$\mathbf{x}_0'=\mathbf{x}_0-\sum^{k-1}_{i=1}(\mathbf{x_0}\cdot\mathbf{v_i})\mathbf{v}_i$$
3) Find $A\mathbf{x}_0'$ then normalise it to get $\mathbf{x_1}$

If the result converges, break out. If not, return to step 2 and continue.

Note that this algorithm does not works for matrix than have repeated eigenvalues. Also, the matrix has to be symmetric.

In [7]:
M = np.array([[4,1,1],[1,2,2],[1,2,4]])
LA.eig(M)

(array([6.13263749, 3.14043537, 0.72692714]),
 array([[ 0.54551277,  0.83043746,  0.11309038],
        [ 0.4683588 , -0.19016213, -0.86283161],
        [ 0.69502219, -0.52365254,  0.49267857]]))

In [8]:
def simple_lanczos(M):
    
    # Initialize lists to store eigenvectors and eigenvalues
    eigenvectors = []
    eigenvalues =[]
    
    # Set tolerance level for convergence
    tol = 1e-8
    
    # Iterate over each eigenvalue/vector
    for i in range(0,M.shape[0]): 
        
        v = np.random.rand(M.shape[0])
        gap = 1
        
        while gap > tol:
            old_v = v
            
            # Orthogonalize v against previous eigenvectors
            a = sum(np.dot(v,vec)* vec for vec in eigenvectors)
            v = v - a
            
            v = np.dot(M,v) 
            
            # Normalize
            v = v / LA.norm(v)
            gap = LA.norm(v - old_v)
            
        eigenvectors.append(v)
        eigenvalues.append((np.dot(M,v)/v)[0])
    
    eigenvectors = np.array(eigenvectors).T
                           
    return (eigenvalues, eigenvectors)

In [9]:
evalues, evectors = simple_lanczos(M)
print("Eigenvalues :", evalues)
print("Eigenvectors :")
print(evectors)

Eigenvalues : [6.132637540357114, 3.1404354093960163, 0.7269272512540828]
Eigenvectors :
[[ 0.54551276 -0.83043747 -0.11309038]
 [ 0.46835881  0.19016212  0.86283162]
 [ 0.6950222   0.52365253 -0.49267856]]


## Lanczos Algorithm

The Lanczos algorithm is an iterative method used for finding a few eigenvalues and eigenvectors of a large, hermitian matrix. 

For matrix $M$:
1) Randomly choose $\mathbf{v}_0$ of length $k$   
2) Initilize matrices $L$ and $H$ so size $k\times k$    
3) Compute $w=Mv_j$, $j$ is the current iteration number   
4) Orthogonalize $w$ against the previous Lanczos vectors $w=w-b_{j-1}v_{j-1}-a_jv_j$     
5) Compute the next Lanczos vector $v_{j+1}$ as the normalized orthogonalized $w$   
6) Upgrade the tridiagonal matrix $H$ with $a$ and $b$

After $k$ iterations, the algorithm generates a tridiagonal matrix $H$, use it to obtain a few eigenvalues and eigenvectors

In [10]:
def lanczos_algorithm(A, k):
    v = np.random.rand(k)
    
    L = np.zeros((k,k),dtype = complex)
    H = np.zeros((k,k),dtype = complex)
    
    L[0] = v/LA.norm(v)
    
    w = np.dot(A,L[0])
    a = np.dot(np.conj(w),L[0])
    w = w- a*L[0]
    H[0,0]=a
    
    for j in range(1,len(v)):
        b = (np.dot(np.conj(w),np.transpose(w)))**0.5
        L[j]=w/b
        
        w = np.dot(A,L[j])
        a = np.dot(np.conj(w) , L[j])
        w = w - a * L[j] - b* L[j-1]
        
        H[j,j]=a
        H[j-1,j]=b
        H[j,j-1]= b
        
    return (H,L)

In [11]:
# Example 
k =3
A = np.array([[4,1,1],[1,2,2],[1,2,4]])

# Compute eigenvalues and eigenvectors using Lanczos algorithm
H,L = lanczos_algorithm(A,k)
lanczos_eigenvalues = LA.eigh(H)[0]
lanczos_eigenvectors = np.dot(LA.eigh(H)[1].T,L).T

# Compute eigenvalues and eigenvectors using numpy
numpy_eigenvalues, numpy_eigenvectors = LA.eigh(A)


# Compare the results
print("Eigenvalues (Lanczos):", lanczos_eigenvalues)
print("Eigenvectors (Lanczos):")
print(lanczos_eigenvectors)
print()

print("Eigenvalues (numpy):", numpy_eigenvalues)
print("Eigenvectors (numpy):")
print(numpy_eigenvectors)

Eigenvalues (Lanczos): [0.72692714 3.14043537 6.13263749]
Eigenvectors (Lanczos):
[[-0.11309038+0.j  0.83043746+0.j -0.54551277+0.j]
 [ 0.86283161+0.j -0.19016213+0.j -0.4683588 +0.j]
 [-0.49267857+0.j -0.52365254+0.j -0.69502219+0.j]]

Eigenvalues (numpy): [0.72692714 3.14043537 6.13263749]
Eigenvectors (numpy):
[[ 0.11309038 -0.83043746 -0.54551277]
 [-0.86283161  0.19016213 -0.4683588 ]
 [ 0.49267857  0.52365254 -0.69502219]]
